In [66]:
import cv2
import numpy as np
import mediapipe as mp
from flask import Flask, render_template, Response
import threading
import base64
from flask_socketio import SocketIO

In [67]:
app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

In [68]:
# Define color ranges in HSV
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 100, 100])
upper_red2 = np.array([180, 255, 255])

lower_green = np.array([35, 100, 100])
upper_green = np.array([85, 255, 255])

lower_blue = np.array([100, 100, 100])
upper_blue = np.array([140, 255, 255])


In [69]:
# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [70]:
cap = cv2.VideoCapture(0)

def generate_frames():
    while True:
        success, frame = cap.read()
        if not success:
            break

        frame = cv2.flip(frame, 1)  # Flip horizontally
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        frame_bgr = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)
        hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)

        mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask_red = cv2.bitwise_or(mask_red1, mask_red2)

        mask_green = cv2.inRange(hsv, lower_green, upper_green)
        mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)

        hand_mask = np.zeros(frame_bgr.shape[:2], dtype=np.uint8)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                hand_points = []
                for landmark in hand_landmarks.landmark:
                    h, w, c = frame_bgr.shape
                    cx, cy = int(landmark.x * w), int(landmark.y * h)
                    hand_points.append((cx, cy))

                hand_points = np.array(hand_points, dtype=np.int32)
                cv2.fillConvexPoly(hand_mask, hand_points, 255)

                for landmark in hand_landmarks.landmark:
                    h, w, c = frame_bgr.shape
                    cx, cy = int(landmark.x * w), int(landmark.y * h)
                    cv2.circle(frame_bgr, (cx, cy), 5, (255, 255, 255), -1)

                mp_drawing.draw_landmarks(frame_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        hand_only = cv2.bitwise_and(frame_bgr, frame_bgr, mask=hand_mask)

        mask_red_hand = cv2.bitwise_and(mask_red, mask_red, mask=hand_mask)
        mask_green_hand = cv2.bitwise_and(mask_green, mask_green, mask=hand_mask)
        mask_blue_hand = cv2.bitwise_and(mask_blue, mask_blue, mask=hand_mask)

        contours_red, _ = cv2.findContours(mask_red_hand, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_green, _ = cv2.findContours(mask_green_hand, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_blue, _ = cv2.findContours(mask_blue_hand, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cv2.drawContours(hand_only, contours_red, -1, (0, 0, 255), 2)
        cv2.drawContours(hand_only, contours_green, -1, (0, 255, 0), 2)
        cv2.drawContours(hand_only, contours_blue, -1, (255, 0, 0), 2)

        _, buffer = cv2.imencode('.jpg', hand_only)
        frame_encoded = base64.b64encode(buffer).decode('utf-8')
        socketio.emit("video_frame", {"frame": frame_encoded})

@app.route('/home')
def home():
    return render_template('index.html')

if __name__ == '__main__':
    thread = threading.Thread(target=generate_frames, daemon=True)
    thread.start()
    socketio.run(app, host='0.0.0.0', port=5000, debug=True, allow_unsafe_werkzeug=True)


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://100.66.223.44:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1